In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, explode, avg, count, desc

In [9]:
spark = SparkSession.builder.appName("movies1").getOrCreate()

In [18]:
movies_file = "/content/movies.csv"
ratings_file = "/content/ratings.csv"

movies_df = spark.read.csv(movies_file, header=True, inferSchema=True)
ratings_df = spark.read.csv(ratings_file, header=True, inferSchema=True)

movies_df.show()
ratings_df.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [11]:
movies_df.createOrReplaceTempView("movies")
ratings_df.createOrReplaceTempView("ratings")

In [12]:
#Find the most active users (users who have rated the most movies).

query1 = """
    SELECT userId, COUNT(movieId) AS count_ratings
    FROM ratings
    GROUP BY userId
    ORDER BY count_ratings DESC
    LIMIT 10
"""

result = spark.sql(query1)
result.show()

+------+-------------+
|userId|count_ratings|
+------+-------------+
|   414|         2698|
|   599|         2478|
|   474|         2108|
|   448|         1864|
|   274|         1346|
|   610|         1302|
|    68|         1260|
|   380|         1218|
|   606|         1115|
|   288|         1055|
+------+-------------+



In [14]:
#Sort the movies name in alphabetic order
query2 = """
  SELECT title
  FROM movies
  ORDER BY title ASC
"""

result = spark.sql(query2)
result.show()

+--------------------+
|               title|
+--------------------+
|"11'09""01 - Sept...|
|          '71 (2014)|
|'Hellboy': The Se...|
|'Round Midnight (...|
| 'Salem's Lot (2004)|
|'Til There Was Yo...|
|'Tis the Season f...|
|  'burbs, The (1989)|
|'night Mother (1986)|
|(500) Days of Sum...|
|*batteries not in...|
|...All the Marble...|
|...And Justice fo...|
|00 Schneider - Ja...|
|   1-900 (06) (1994)|
|           10 (1979)|
|10 Cent Pistol (2...|
|10 Cloverfield La...|
|10 Items or Less ...|
|10 Things I Hate ...|
+--------------------+
only showing top 20 rows



In [20]:
#Calculate the average rating per genre
movies_split_genres_df = movies_df.withColumn("IndividualGenre", explode(split(col("genres"), "\\|")))
movies_split_genres_df = movies_split_genres_df.drop("genres")
movies_split_genres_df.show()

+-------+--------------------+---------------+
|movieId|               title|IndividualGenre|
+-------+--------------------+---------------+
|      1|    Toy Story (1995)|      Adventure|
|      1|    Toy Story (1995)|      Animation|
|      1|    Toy Story (1995)|       Children|
|      1|    Toy Story (1995)|         Comedy|
|      1|    Toy Story (1995)|        Fantasy|
|      2|      Jumanji (1995)|      Adventure|
|      2|      Jumanji (1995)|       Children|
|      2|      Jumanji (1995)|        Fantasy|
|      3|Grumpier Old Men ...|         Comedy|
|      3|Grumpier Old Men ...|        Romance|
|      4|Waiting to Exhale...|         Comedy|
|      4|Waiting to Exhale...|          Drama|
|      4|Waiting to Exhale...|        Romance|
|      5|Father of the Bri...|         Comedy|
|      6|         Heat (1995)|         Action|
|      6|         Heat (1995)|          Crime|
|      6|         Heat (1995)|       Thriller|
|      7|      Sabrina (1995)|         Comedy|
|      7|    

In [23]:
movies_split_genres_df.createOrReplaceTempView("movies_split_genres")

query3 = """
  SELECT m.IndividualGenre, AVG(r.rating) AS avg_rating
  FROM movies_split_genres m
  JOIN
  ratings r
  ON m.movieId = r.movieId
  GROUP BY m.IndividualGenre
  ORDER BY avg_rating DESC
"""

result = spark.sql(query3)
result.show()

+------------------+------------------+
|   IndividualGenre|        avg_rating|
+------------------+------------------+
|         Film-Noir| 3.920114942528736|
|               War|   3.8082938876312|
|       Documentary| 3.797785069729286|
|             Crime| 3.658293867274144|
|             Drama|3.6561844113718758|
|           Mystery| 3.632460255407871|
|         Animation|3.6299370349170004|
|              IMAX| 3.618335343787696|
|           Western| 3.583937823834197|
|           Musical|3.5636781053649105|
|         Adventure|3.5086089151939075|
|           Romance|3.5065107040388437|
|          Thriller|3.4937055799183425|
|           Fantasy|3.4910005070136894|
|(no genres listed)|3.4893617021276597|
|            Sci-Fi| 3.455721162210752|
|            Action| 3.447984331646809|
|          Children| 3.412956125108601|
|            Comedy|3.3847207640898267|
|            Horror| 3.258195034974626|
+------------------+------------------+



In [24]:
spark.stop()